In [1]:
cd /scratch/xc1490/projects/ecog/ALAE_1023/neural_speech_decoding

/scratch/xc1490/projects/ecog/ALAE_1023/neural_speech_decoding


In [2]:
LOAD = 1

import torch
from torch import optim as optim
import torch.utils.data
from tqdm import tqdm as tqdm
import numpy as np
import argparse, os, json, yaml
from networks import *
from model import Model
from dataset import *
from tracker import LossTracker
from utils.custom_adam import LREQAdam
from utils.checkpointer import Checkpointer
from utils.launcher import run
from utils.defaults import get_cfg_defaults
from utils.save import save_sample
device = "cuda" if torch.cuda.is_available() else "cpu"

cd /scratch/xc1490/projects/ecog/ALAE_1023/neural_speech_decoding
python train_e2a.py --OUTPUT_DIR output/resnet_NY869 --trainsubject NY869 --testsubject NY869 --param_file configs/e2a_production.yaml --batch_size 16 --MAPPING_FROM_ECOG ECoGMapping_ResNet --reshape 1 --DENSITY "LD" --wavebased 1 --dynamicfiltershape 0 --n_filter_samples 80 --n_fft 512 --formant_supervision 1  --intensity_thres -1 --epoch_num 60 --pretrained_model_dir output/a2a/NY869 --causal 0

In [3]:
pwd

'/scratch/xc1490/projects/ecog/ALAE_1023/neural_speech_decoding'

In [4]:
class Args:
  noise_db = -50
    
  opts = None
  config_file = 'configs/e2a_production.yaml'
  DENSITY = 'LD'
  wavebased = 1
  bgnoise_fromdata = 1
  ignore_loading = 0
  finetune = 0
  learnedmask = 0
  dynamicfiltershape = 0
  formant_supervision = 0
  pitch_supervision = 0
  intensity_supervision = 0
  n_filter_samples = 20
  n_fft = 512
  reverse_order = 1
  lar_cap = 0
  intensity_thres = -1
  RNN_COMPUTE_DB_LOUDNESS = 1
  BIDIRECTION = 1
  MAPPING_FROM_ECOG = 'ECoGMapping_ResNet'
  OUTPUT_DIR = 'output/resnet'
  COMPONENTKEY = ''
  trainsubject = 'NY869'
  testsubject = 'NY869'
  reshape = -1
  ld_loss_weight = 1
  alpha_loss_weight = 1
  consonant_loss_weight = 0
  batch_size = 8
  param_file = 'configs/e2a_production.yaml'
  pretrained_model_dir = 'output/a2a/NY869'
  causal = 0
  anticausal = 0
  rdropout = 0
  epoch_num = 100
  use_stoi = 0
  use_denoise = 0

args_ = Args()

In [5]:
with open("configs/AllSubjectInfo.json", "r") as rfile:
    allsubj_param = json.load(rfile)
with open(args_.param_file, 'r') as stream:
    param = yaml.safe_load(stream)
(
    ecog_all,
    wave_orig_all,
    x_orig_all,
    x_orig_amp_all,
    labels_all,
    gender_train_all,
    on_stage_all,
    on_stage_wider_all
) = ({}, {}, {}, {}, {}, {}, {}, {})

hann_win = torch.hann_window(21, periodic=False).reshape([1, 1, 21, 1])
hann_win = hann_win / hann_win.sum()

In [8]:
def get_train_data(
    ecog_all,
    wave_orig_all,
    x_orig_all,
    x_orig_amp_all,
    labels_all,
    gender_train_all,
    on_stage_all,
    on_stage_wider_all,
    sample_dict_train=None,
    subject=None,
):
    wave_orig_all[subject] = (
        sample_dict_train["wave_re_batch_all"].to(device).float()
    )
    gender_train_all[subject] =sample_dict_train['gender_all'].to(device).float()
    if cfg.MODEL.WAVE_BASED:
        x_orig_all[subject] = (
            sample_dict_train["wave_spec_re_batch_all"].to(device).float()
        )
        x_orig_amp_all[subject] = (
            sample_dict_train["wave_spec_re_amp_batch_all"].to(device).float()
        )
    on_stage_all[subject] = (
        sample_dict_train["on_stage_re_batch_all"].to(device).float()
    )
    on_stage_wider_all[subject] = (
        sample_dict_train["on_stage_wider_re_batch_all"].to(device).float()
    )
    labels_all[subject] = sample_dict_train["label_batch_all"]
    ecog_all[subject] = sample_dict_train["ecog_re_batch_all"].to(device).float()

    return (
        ecog_all,
        wave_orig_all,
        x_orig_all,
        x_orig_amp_all,
        labels_all,
        gender_train_all,
        on_stage_all,
        on_stage_wider_all,
        )

In [9]:
def load_model_checkpoint(
    logger,
    local_rank,
    distributed,
    tracker=None,
    tracker_test=None,
    dataset_all=None,
    subject="NY742",
    load_dir="",
    single_patient_mapping=0,param=None
):
    if args_.trainsubject != "":
        train_subject_info = args_.trainsubject.split(",")
    else:
        train_subject_info = cfg.DATASET.SUBJECT
    model = Model(
        generator=cfg.MODEL.GENERATOR,
        encoder=cfg.MODEL.ENCODER,
        ecog_encoder_name=cfg.MODEL.MAPPING_FROM_ECOG,
        spec_chans=cfg.DATASET.SPEC_CHANS,
        n_formants=cfg.MODEL.N_FORMANTS,
        n_formants_noise=cfg.MODEL.N_FORMANTS_NOISE,
        n_formants_ecog=cfg.MODEL.N_FORMANTS_ECOG,
        wavebased=cfg.MODEL.WAVE_BASED,
        n_fft=cfg.MODEL.N_FFT,
        noise_db=cfg.MODEL.NOISE_DB,
        max_db=cfg.MODEL.MAX_DB,
        with_ecog=cfg.MODEL.ECOG,
        do_mel_guide=cfg.MODEL.DO_MEL_GUIDE,
        noise_from_data=cfg.MODEL.BGNOISE_FROMDATA and cfg.DATASET.PROD,
        specsup=cfg.FINETUNE.SPECSUP,
        power_synth=cfg.MODEL.POWER_SYNTH,
        apply_flooding=cfg.FINETUNE.APPLY_FLOODING,
        normed_mask=cfg.MODEL.NORMED_MASK,
        dummy_formant=cfg.MODEL.DUMMY_FORMANT,
        A2A=cfg.VISUAL.A2A,
        causal=cfg.MODEL.CAUSAL,
        anticausal=cfg.MODEL.ANTICAUSAL,
        pre_articulate=cfg.DATASET.PRE_ARTICULATE,
        alpha_sup=param["Subj"][subject][
            "AlphaSup"
        ],
        ld_loss_weight=cfg.MODEL.ld_loss_weight,
        alpha_loss_weight=cfg.MODEL.alpha_loss_weight,
        consonant_loss_weight=cfg.MODEL.consonant_loss_weight,
        component_regression=cfg.MODEL.component_regression,
        amp_formant_loss_weight=cfg.MODEL.amp_formant_loss_weight,
        freq_single_formant_loss_weight=cfg.MODEL.freq_single_formant_loss_weight,
        amp_minmax=cfg.MODEL.amp_minmax,
        amp_energy=cfg.MODEL.amp_energy,
        f0_midi=cfg.MODEL.f0_midi,
        alpha_db=cfg.MODEL.alpha_db,
        network_db=cfg.MODEL.network_db,
        consistency_loss=cfg.MODEL.consistency_loss,
        delta_time=cfg.MODEL.delta_time,
        delta_freq=cfg.MODEL.delta_freq,
        cumsum=cfg.MODEL.cumsum,
        distill=cfg.MODEL.distill,
        learned_mask=cfg.MODEL.LEARNED_MASK,
        n_filter_samples=cfg.MODEL.N_FILTER_SAMPLES,
        patient=subject,
        batch_size=cfg.TRAIN.BATCH_SIZE,
        rdropout=cfg.MODEL.rdropout,
        dynamic_filter_shape=cfg.MODEL.DYNAMIC_FILTER_SHAPE,
        learnedbandwidth=cfg.MODEL.LEARNEDBANDWIDTH,
        gender_patient=allsubj_param["Subj"][train_subject_info[0]]["Gender"],
        reverse_order=args_.reverse_order,
        larger_capacity=args_.lar_cap,
        use_stoi=args_.use_stoi,
    )

    if torch.cuda.is_available():
        model.cuda(local_rank)
    model.train()

    model_s = Model(
        generator=cfg.MODEL.GENERATOR,
        encoder=cfg.MODEL.ENCODER,
        ecog_encoder_name=cfg.MODEL.MAPPING_FROM_ECOG,
        spec_chans=cfg.DATASET.SPEC_CHANS,
        n_formants=cfg.MODEL.N_FORMANTS,
        n_formants_noise=cfg.MODEL.N_FORMANTS_NOISE,
        n_formants_ecog=cfg.MODEL.N_FORMANTS_ECOG,
        wavebased=cfg.MODEL.WAVE_BASED,
        n_fft=cfg.MODEL.N_FFT,
        noise_db=cfg.MODEL.NOISE_DB,
        max_db=cfg.MODEL.MAX_DB,
        with_ecog=cfg.MODEL.ECOG,
        do_mel_guide=cfg.MODEL.DO_MEL_GUIDE,
        noise_from_data=cfg.MODEL.BGNOISE_FROMDATA and cfg.DATASET.PROD,
        specsup=cfg.FINETUNE.SPECSUP,
        power_synth=cfg.MODEL.POWER_SYNTH,
        apply_flooding=cfg.FINETUNE.APPLY_FLOODING,
        normed_mask=cfg.MODEL.NORMED_MASK,
        dummy_formant=cfg.MODEL.DUMMY_FORMANT,
        A2A=cfg.VISUAL.A2A,
        causal=cfg.MODEL.CAUSAL,
        anticausal=cfg.MODEL.ANTICAUSAL,
        pre_articulate=cfg.DATASET.PRE_ARTICULATE,
        alpha_sup=param["Subj"][subject][
            "AlphaSup"
        ],
        ld_loss_weight=cfg.MODEL.ld_loss_weight,
        alpha_loss_weight=cfg.MODEL.alpha_loss_weight,
        consonant_loss_weight=cfg.MODEL.consonant_loss_weight,
        component_regression=cfg.MODEL.component_regression,
        amp_formant_loss_weight=cfg.MODEL.amp_formant_loss_weight,
        freq_single_formant_loss_weight=cfg.MODEL.freq_single_formant_loss_weight,
        amp_minmax=cfg.MODEL.amp_minmax,
        amp_energy=cfg.MODEL.amp_energy,
        f0_midi=cfg.MODEL.f0_midi,
        alpha_db=cfg.MODEL.alpha_db,
        network_db=cfg.MODEL.network_db,
        consistency_loss=cfg.MODEL.consistency_loss,
        delta_time=cfg.MODEL.delta_time,
        delta_freq=cfg.MODEL.delta_freq,
        cumsum=cfg.MODEL.cumsum,
        distill=cfg.MODEL.distill,
        learned_mask=cfg.MODEL.LEARNED_MASK,
        n_filter_samples=cfg.MODEL.N_FILTER_SAMPLES,
        patient=subject,
        batch_size=cfg.TRAIN.BATCH_SIZE,
        rdropout=cfg.MODEL.rdropout,
        dynamic_filter_shape=cfg.MODEL.DYNAMIC_FILTER_SHAPE,
        learnedbandwidth=cfg.MODEL.LEARNEDBANDWIDTH,
        gender_patient=allsubj_param["Subj"][train_subject_info[0]]["Gender"],
        reverse_order=args_.reverse_order,
        larger_capacity=args_.lar_cap,
        use_stoi=args_.use_stoi,
    )
    if torch.cuda.is_available():
        model_s.cuda(local_rank)
    model_s.eval()
    model_s.requires_grad_(False)
    if distributed:
        model = nn.parallel.DistributedDataParallel(
            model,
            device_ids=[local_rank],
            broadcast_buffers=False,
            bucket_cap_mb=25,
            find_unused_parameters=True,
        )
        model.device_ids = None
        decoder = model.module.decoder
        encoder = model.module.encoder
        if hasattr(model.module, "ecog_encoder"):
            ecog_encoder = model.module.ecog_encoder
            if torch.cuda.is_available():
                ecog_encoder = ecog_encoder.cuda(local_rank)
            # ecog_encoder.performer.cuda(local_rank)
        if hasattr(model.module, "decoder_mel"):
            decoder_mel = model.module.decoder_mel
    else:
        decoder = model.decoder
        encoder = model.encoder
        if hasattr(model, "ecog_encoder"):
            ecog_encoder = model.ecog_encoder
            if torch.cuda.is_available():
                ecog_encoder = ecog_encoder.cuda(local_rank)
        if hasattr(model, "decoder_mel"):
            decoder_mel = model.decoder_mel
    logger.info("Trainable parameters generator:")
    logger.info("Trainable parameters discriminator:")
    arguments = dict()
    arguments["iteration"] = 0

    if cfg.MODEL.ECOG:
        if cfg.MODEL.SUPLOSS_ON_ECOGF:
            optimizer = LREQAdam(
                [{"params": ecog_encoder.parameters()}],
                lr=cfg.TRAIN.BASE_LEARNING_RATE,
                betas=(cfg.TRAIN.ADAM_BETA_0, cfg.TRAIN.ADAM_BETA_1),
                weight_decay=0,
            )
        else:
            optimizer = LREQAdam(
                [
                    {"params": ecog_encoder.parameters()},
                    {"params": decoder.parameters()},
                ],
                lr=cfg.TRAIN.BASE_LEARNING_RATE,
                betas=(cfg.TRAIN.ADAM_BETA_0, cfg.TRAIN.ADAM_BETA_1),
                weight_decay=0,
            )

    else:
        if cfg.MODEL.DO_MEL_GUIDE:
            optimizer = LREQAdam(
                [
                    {"params": encoder.parameters()},
                    {"params": decoder.parameters()},
                    {"params": decoder_mel.parameters()},
                ],
                lr=cfg.TRAIN.BASE_LEARNING_RATE,
                betas=(cfg.TRAIN.ADAM_BETA_0, cfg.TRAIN.ADAM_BETA_1),
                weight_decay=0,
            )
        else:
            optimizer = LREQAdam(
                [{"params": encoder.parameters()}, {"params": decoder.parameters()}],
                lr=cfg.TRAIN.BASE_LEARNING_RATE,
                betas=(cfg.TRAIN.ADAM_BETA_0, cfg.TRAIN.ADAM_BETA_1),
                weight_decay=0,
            )
    model_dict = {
        "encoder": encoder,
        "generator": decoder,
    }
    if hasattr(model, "ecog_encoder"):
        model_dict["ecog_encoder"] = ecog_encoder
    if hasattr(model, "decoder_mel"):
        model_dict["decoder_mel"] = decoder_mel
    if local_rank == 0:
        model_dict["encoder_s"] = model_s.encoder.to(device)
        model_dict["generator_s"] = model_s.decoder.to(device)
        if hasattr(model_s, "ecog_encoder"):
            model_dict["ecog_encoder_s"] = model_s.ecog_encoder.to(device)
        if hasattr(model_s, "decoder_mel"):
            model_dict["decoder_mel_s"] = model_s.decoder_mel
    noise_dist = torch.from_numpy(dataset_all[subject].noise_dist).to(device).float()
    if cfg.MODEL.BGNOISE_FROMDATA:
        model_s.noise_dist_init(noise_dist)
        model.noise_dist_init(noise_dist)
    if cfg.MODEL.ECOG:
        if cfg.MODEL.SUPLOSS_ON_ECOGF:
            optimizer = LREQAdam(
                [{"params": ecog_encoder.parameters()}],
                lr=cfg.TRAIN.BASE_LEARNING_RATE,
                betas=(cfg.TRAIN.ADAM_BETA_0, cfg.TRAIN.ADAM_BETA_1),
                weight_decay=0,
            )
        else:
            optimizer = LREQAdam(
                [
                    {"params": ecog_encoder.parameters()},
                    {"params": decoder.parameters()},
                ],
                lr=cfg.TRAIN.BASE_LEARNING_RATE,
                betas=(cfg.TRAIN.ADAM_BETA_0, cfg.TRAIN.ADAM_BETA_1),
                weight_decay=0,
            )
    else:
        if cfg.MODEL.DO_MEL_GUIDE:
            optimizer = LREQAdam(
                [
                    {"params": encoder.parameters()},
                    {"params": decoder.parameters()},
                    {"params": decoder_mel.parameters()},
                ],
                lr=cfg.TRAIN.BASE_LEARNING_RATE,
                betas=(cfg.TRAIN.ADAM_BETA_0, cfg.TRAIN.ADAM_BETA_1),
                weight_decay=0,
            )
        else:
            optimizer = LREQAdam(
                [
                    {"params": encoder.parameters()},
                    {"params": decoder.parameters()},
                ],
                lr=cfg.TRAIN.BASE_LEARNING_RATE,
                betas=(cfg.TRAIN.ADAM_BETA_0, cfg.TRAIN.ADAM_BETA_1),
                weight_decay=0,
            )
    tracker = LossTracker(cfg.OUTPUT_DIR)
    tracker_test = LossTracker(cfg.OUTPUT_DIR, test=True)
    auxiliary = {
        "optimizer": optimizer,
        "tracker": tracker,
        "tracker_test": tracker_test,
    }
    checkpointer = Checkpointer(
        cfg, model_dict, auxiliary, logger=logger, save=local_rank == 0
    )
    if LOAD:
        extra_checkpoint_data = checkpointer.load(
            ignore_last_checkpoint=True if LOAD!=0 else False,
            ignore_auxiliary=True,
            file_name=load_dir,
        )
        arguments.update(extra_checkpoint_data)
    return (
        checkpointer,
        model,
        model_s,
        encoder,
        decoder,
        ecog_encoder,
        optimizer,
        tracker,
        tracker_test,
    )

In [10]:
def train(cfg, logger, local_rank, world_size, distributed):
    if torch.cuda.is_available():
        torch.cuda.set_device(local_rank)
    with open('configs/train_param_production.json', 'r') as stream:
        param = json.load(stream)
    dataset_all, dataset_test_all = {}, {}
    train_subject_info = args_.trainsubject.split(",") if args_.trainsubject != "" else cfg.DATASET.SUBJECT 
    test_subject_info = args_.testsubject.split(",") if args_.testsubject != "" else cfg.DATASET.SUBJECT

    for subject in np.union1d(train_subject_info, test_subject_info):
        dataset_all[subject] = TFRecordsDataset(
            cfg,
            logger,
            rank=local_rank,
            world_size=world_size,
            SUBJECT=[subject],
            buffer_size_mb=1024,
            channels=cfg.MODEL.CHANNELS,
            param=param,
            allsubj_param=allsubj_param,
            ReshapeAsGrid=1,
            rearrange_elec=0,
            low_density=cfg.DATASET.DENSITY == "LD",
            process_ecog=True,
        )

    for subject in test_subject_info:
        dataset_test_all[subject] = TFRecordsDataset(
            cfg,
            logger,
            rank=local_rank,
            world_size=world_size,
            SUBJECT=[subject],
            buffer_size_mb=1024,
            channels=cfg.MODEL.CHANNELS,
            train=False,
            param=param,
            allsubj_param=allsubj_param,
            ReshapeAsGrid=1,
            rearrange_elec=0,
            low_density=cfg.DATASET.DENSITY == "LD",
            process_ecog=True,
        )
    tracker = LossTracker(cfg.OUTPUT_DIR)
    tracker_test = LossTracker(cfg.OUTPUT_DIR, test=True)

    (checkpointer_all,
        model_all,
        model_s_all,
        encoder_all,
        decoder_all,
        ecog_encoder_all,
        optimizer_all,
    ) = ({}, {}, {}, {}, {}, {}, {})

    for single_patient_mapping, subject in enumerate(
        np.union1d(train_subject_info, test_subject_info)
    ): 
        if args_.pretrained_model_dir != "":
            load_sub_dir = args_.pretrained_model_dir
            max_epoch = (
                np.array(
                    [
                        i.split('epoch')[1].split('.pth')[0]
                        for i in os.listdir(load_sub_dir)
                        if i.endswith("pth")
                    ]
                )
                .astype("int")
                .max()
            )
            load_sub_name = [i for i in load_sub_dir.split("/") if "NY" in i][0]
            print("subject, load_sub_name", subject, load_sub_name)
            load_sub_dir = load_sub_dir + "/{}_a2a_model_epoch{}.pth".format(
                   load_sub_name, max_epoch  
                )
            print("pretrained load dir", load_sub_dir)
        else:
            load_sub_dir = ''
            print ('No pretrainde a2a model provided!')
            #raise Exception("Please Provide pretrained_model_dir")
        (
            checkpointer_all[subject],
            model_all[subject],
            model_s_all[subject],
            encoder_all[subject],
            decoder_all[subject],
            ecog_encoder_all[subject],
            optimizer_all[subject],
            tracker,
            tracker_test,
        ) = load_model_checkpoint(
            logger,
            local_rank,
            distributed,
            tracker=tracker,
            tracker_test=tracker_test,
            dataset_all=dataset_all,
            subject=subject,
            load_dir=load_sub_dir,
            single_patient_mapping=single_patient_mapping,param=param
        )
    loadsub = train_subject_info[0]
    ecog_encoder_shared = ecog_encoder_all[loadsub]

    for single_patient_mapping, subject in enumerate(
        np.union1d(train_subject_info, test_subject_info)
    ):
        model_all[
            subject
        ].ecog_encoder = ecog_encoder_shared
        model_s_all[
            subject
        ].ecog_encoder = ecog_encoder_shared
    (   
        ecog_test_all,
        sample_wave_test_all,
        sample_spec_test_all,
        sample_spec_amp_test_all,
        sample_label_test_all,
        gender_test_all,
        on_stage_test_all,
        on_stage_wider_test_all,
    ) = (
        {},{},{},{},{},{},{},{})

    hann_win = torch.hann_window(21, periodic=False).reshape([1, 1, 21, 1])
    hann_win = hann_win / hann_win.sum()
    x_amp_from_denoise = False

    for subject in test_subject_info:
        dataset_test_all[subject].reset(
            cfg.DATASET.MAX_RESOLUTION_LEVEL, len(dataset_test_all[subject].dataset)
        )
        sample_dict_test = next(iter(dataset_test_all[subject].iterator))
        gender_test_all[subject] = sample_dict_test['gender_all'].to(device).float()
        if cfg.DATASET.PROD:
            sample_wave_test_all[subject] = (
                sample_dict_test["wave_re_batch_all"].to(device).float()
            )
            if cfg.MODEL.WAVE_BASED:
                sample_spec_test_all[subject] = (
                    sample_dict_test["wave_spec_re_batch_all"].to(device).float()
                )
                sample_spec_amp_test_all[subject] = (
                    sample_dict_test["wave_spec_re_amp_batch_all"]
                    .to(device)
                    .float()
                )
            sample_label_test_all[subject] = sample_dict_test["label_batch_all"]
            if cfg.MODEL.ECOG:
                ecog_test_all[subject] = sample_dict_test["ecog_re_batch_all"].to(device).float()
            on_stage_test_all[subject] = (
                sample_dict_test["on_stage_re_batch_all"].to(device).float()
            )
            on_stage_wider_test_all[subject] = (
                sample_dict_test["on_stage_wider_re_batch_all"].to(device).float()
            )
    duomask = True
    x_amp_from_denoise = False
    n_iter = 0

    (
        ecog_all,
        wave_orig_all,
        x_orig_all,
        x_orig_amp_all,
        labels_all,
        gender_train_all,
        on_stage_all,
        on_stage_wider_all
    ) = (
        {},{},{},{},{},{},{},{}
    )
    for epoch in tqdm(range(cfg.TRAIN.TRAIN_EPOCHS)):
        
        
        #train
        for subject in train_subject_info:
            model_all[subject].train()
        i = 0
        dataset_iterator_all = {}
        if len(train_subject_info) <= 1:
            dataset_iterator_all[train_subject_info[0]] = iter(
                dataset_all[train_subject_info[0]].iterator
            )
            sample_dict_train_all = {}
            for sample_dict_train_all[train_subject_info[0]] in tqdm(
                dataset_iterator_all[train_subject_info[0]]
            ):
                n_iter += 1
                i += 1
                for subject in train_subject_info:
                    if n_iter % 200 == 0:
                        print(tracker.register_means(n_iter))
                    (
                        ecog_all,
                        wave_orig_all,
                        x_orig_all,
                        x_orig_amp_all,
                        labels_all,
                        gender_train_all,
                        on_stage_all,
                        on_stage_wider_all
                    ) = get_train_data(
                        ecog_all,
                        wave_orig_all,
                        x_orig_all,
                        x_orig_amp_all,
                        labels_all,
                        gender_train_all,
                        on_stage_all,
                        on_stage_wider_all,
                        sample_dict_train_all[train_subject_info[0]],
                        subject=subject,
                    )
                    initial = None
                    
                    optimizer_all[subject].zero_grad()
                    Lrec, tracker = model_all[subject](
                        x_orig_all[subject],
                        ecog=ecog_all[subject],
                        on_stage=on_stage_all[subject],
                        on_stage_wider=on_stage_all[subject],
                        ae=False,
                        tracker=tracker,
                        encoder_guide=cfg.MODEL.W_SUP,
                        duomask=duomask,
                        x_amp=x_orig_amp_all[subject],
                        x_amp_from_denoise=x_amp_from_denoise,
                        gender=gender_train_all[subject],
                    )
                    (Lrec).backward()
                    optimizer_all[subject].step()

                    betta = 0.5 ** (cfg.TRAIN.BATCH_SIZE / (10 * 1000.0))
                    model_s_all[subject].lerp(
                        model_all[subject],
                        betta,
                        w_classifier=cfg.MODEL.W_CLASSIFIER,
                    )

        #test
        for subject in test_subject_info:
            print(
                2
                ** (
                    torch.tanh(
                        model_all[subject].encoder.formant_bandwitdh_slop
                    )
                )
            )
            print("save test result!")

            model_all[subject].eval()
            Lrec = model_all[subject](
                sample_spec_test_all[subject],
                x_denoise=None,
                x_mel=None,
                ecog=ecog_test_all[subject] if cfg.MODEL.ECOG else None,
                on_stage=on_stage_test_all[subject],
                ae=not cfg.MODEL.ECOG,
                tracker=tracker_test,
                encoder_guide=cfg.MODEL.W_SUP,
                pitch_aug=False,
                duomask=duomask,
                debug=False,
                x_amp=sample_spec_amp_test_all[subject],
                hamonic_bias=False,
                gender=gender_test_all[subject],
                on_stage_wider=on_stage_test_all[subject],
            )

            initial = None

            if epoch % 1 == 0:
                checkpointer_all[subject].save(
                    "model_epoch{}_{}".format(epoch, subject)
                )
                save_sample(
                    cfg,
                    sample_spec_test_all[subject],
                    ecog_test_all[subject],
                    encoder_all[subject],
                    decoder_all[subject],
                    ecog_encoder_shared
                    if hasattr(model_all[subject], "ecog_encoder")
                    else None,
                    encoder2
                    if hasattr(model_all[subject], "encoder2")
                    else None,
                    x_denoise=None,
                    decoder_mel=decoder_mel if cfg.MODEL.DO_MEL_GUIDE else None,
                    epoch=epoch,
                    label=sample_label_test_all[subject],
                    mode="test",
                    path=cfg.OUTPUT_DIR,
                    tracker=tracker_test,
                    linear=cfg.MODEL.WAVE_BASED,
                    n_fft=cfg.MODEL.N_FFT,
                    duomask=duomask,
                    x_amp=sample_spec_amp_test_all[subject],
                    gender=gender_test_all[subject],
                    sample_wave=sample_wave_test_all[subject],
                    sample_wave_denoise=None,
                    on_stage_wider=on_stage_test_all[subject],
                    auto_regressive=False,
                    seq_out_start=initial,
                    suffix=subject,
                )

        

In [11]:
gpu_count = torch.cuda.device_count()
cfg = get_cfg_defaults()
if args_.trainsubject != "":
    train_subject_info = args_.trainsubject.split(",")
else:
    train_subject_info = cfg.DATASET.SUBJECT
if args_.testsubject != "":
    test_subject_info = args_.testsubject.split(",")
else:
    test_subject_info = cfg.DATASET.SUBJECT
with open("configs/AllSubjectInfo.json", "r") as rfile:
    allsubj_param = json.load(rfile)
subj_param = allsubj_param["Subj"][args_.trainsubject.split(",")[0]]
Gender = subj_param["Gender"] if cfg.DATASET.PROD else "Female"
config_file = args_.param_file
cfg.merge_from_file(config_file)
args_.config_file = config_file

In [12]:
run(
    train,
    cfg,
    description="StyleGAN",
    default_config=config_file,
    world_size=gpu_count,
    args_=args_,
)

rank in _run 0
2023-06-11 00:17:42,576 logger INFO: <__main__.Args object at 0x14c7020cc850>
2023-06-11 00:17:42,579 logger INFO: World size: 1
2023-06-11 00:17:42,580 logger INFO: Loaded configuration file configs/e2a_production.yaml
TestNum_cum 1
ecog_alldataset 1
end_ind_re_valid_alldataset 1
formant_re_alldataset 1
intensity_re_alldataset 1
label_alldataset 1
noisesample_re_alldataset 1
pitch_re_alldataset 1
start_ind_re_valid_alldataset 1
wave_re_alldataset 1
wave_re_spec_alldataset 1
wave_re_spec_amp_alldataset 1
self.meta_data[ TestNum_cum s] [50]
dict_keys(['TestNum_cum', 'ecog_alldataset', 'end_ind_re_valid_alldataset', 'formant_re_alldataset', 'intensity_re_alldataset', 'label_alldataset', 'noisesample_re_alldataset', 'pitch_re_alldataset', 'start_ind_re_valid_alldataset', 'wave_re_alldataset', 'wave_re_spec_alldataset', 'wave_re_spec_amp_alldataset'])
self.ReshapeAsGrid:  1 ECoGMapping_ResNet
dict_keys(['TestNum_cum', 'ecog_alldataset', 'end_ind_re_valid_alldataset', 'forman

  0%|          | 0/47 [00:00<?, ?it/s]

length, self.TestNum_cum [50]
length, self.TestNum_cum [50]
length, self.TestNum_cum [50]



100%|██████████| 47/47 [01:33<00:00,  1.98s/it]


length, self.TestNum_cum [50]
tensor([1.])
save test result!
2023-06-11 00:19:21,759 logger INFO: Saving checkpoint to output/resnet/model_epoch0_NY869.pth
registering means,n_iter,self.n_iters 0 []
0 Lae_a1 0.0
0 Lae_a_l21 0.0
0 Lae_db1 0.2677802
0 Lae_db_l21 0.267818
0 Lae_a2 0.0
0 Lae_a_l22 0.0
0 Lae_db2 0.26032233
0 Lae_db_l22 0.26033404
0 Lrec 21.124102
0 loudness_metric 0.015242103
0 loudness 4.00646
0 f0_metric 1.9358493
0 f0_hz 0.5807548
0 amplitudes_metric 0.013717631
0 amplitudes 6.700783
0 amplitude_formants_hamon_metric 0.0007394142
0 amplitude_formants_hamon 0.29576567
0 freq_formants_hamon_hz_metric_2 0.11886589
0 freq_formants_hamon_hz_metric_6 0.1609572
0 freq_formants_hamon 0.37046427
0 amplitude_formants_noise_metric 0.0019222703
0 amplitude_formants_noise 0.76890814
0 freq_formants_noise_metric 0.4208961
0 freq_formants_noise 1.9831544
0 bandwidth_formants_noise_hz_metric 2.548035
0 bandwidth_formants_noise_hz 7.644105
0 Ldiff 0.04359938
0 Lexp -0.16299137


...


100%|██████████| 47/47 [01:30<00:00,  1.93s/it]


length, self.TestNum_cum [50]
tensor([1.])
save test result!
2023-06-11 00:47:53,555 logger INFO: Saving checkpoint to output/resnet/model_epoch8_NY869.pth
registering means,n_iter,self.n_iters 8 [0, 1, 2, 3, 4, 5, 6, 7]
8 Lae_a1 0.0
8 Lae_a_l21 0.0
8 Lae_db1 0.22700496
8 Lae_db_l21 0.22705537
8 Lae_a2 0.0
8 Lae_a_l22 0.0
8 Lae_db2 0.22505563
8 Lae_db_l22 0.22506937
8 Lrec 18.082424
8 loudness_metric 0.013084595
8 loudness 3.1863556
8 f0_metric 0.09715257
8 f0_hz 0.029145772
8 amplitudes_metric 0.011779965
8 amplitudes 6.414452
8 amplitude_formants_hamon_metric 0.00062535494
8 amplitude_formants_hamon 0.25014198
8 freq_formants_hamon_hz_metric_2 0.14478149
8 freq_formants_hamon_hz_metric_6 0.11209976
8 freq_formants_hamon 0.25550583
8 amplitude_formants_noise_metric 0.0021536024
8 amplitude_formants_noise 0.86144096
8 freq_formants_noise_metric 0.36376724
8 freq_formants_noise 1.7274042
8 bandwidth_formants_noise_hz_metric 2.2072618
8 bandwidth_formants_noise_hz 6.621785
8 Ldiff 0.0250

  8%|▊         | 8/100 [31:59<6:07:56, 239.97s/it]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/ext3/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-b13e55f54dc2>", line 7, in <module>
    args_=args_,
  File "/scratch/xc1490/projects/ecog/ALAE_1023/neural_speech_decoding/utils/launcher.py", line 174, in run
    _run(0, world_size, fn, defaults, write_log, no_cuda, args_)
  File "/scratch/xc1490/projects/ecog/ALAE_1023/neural_speech_decoding/utils/launcher.py", line 158, in _run
    fn(**matching_args_)
  File "<ipython-input-10-a02f4c11cfd6>", line 309, in train
    suffix=subject,
  File "/scratch/xc1490/projects/ecog/ALAE_1023/neural_speech_decoding/utils/save.py", line 981, in save_sample
    129,
  File "/scratch/xc1490/projects/ecog/ALAE_1023/neural_speech_decoding/utils/save.py", line 117, in mygriffinlim
    wave_gt[i] = griffinlim(msgram[i] ** 0.5, hop_length=hop_length)
  File "/ext3/miniconda3/lib/p

TypeError: object of type 'NoneType' has no len()

In [5]:
rm data/NY869.h5 

In [6]:
#!rsync ../data/data/LD_data_extracted/meta_data/NY869.h5 data/
#mv data/NY869.h5 data/NY869_full.h5
from tqdm.notebook import tqdm
meta_data = h5py.File('data/NY869_full.h5')
keys_to_store = ['ecog_alldataset','label_alldataset',\
'formant_re_alldataset','pitch_re_alldataset','intensity_re_alldataset',\
'start_ind_re_valid_alldataset','end_ind_re_valid_alldataset','wave_re_spec_alldataset',\
'wave_re_alldataset','wave_re_spec_amp_alldataset','noisesample_re_alldataset']
with h5py.File('data/NY869.h5' , 'w') as hf:
    for key in tqdm(keys_to_store):
        print (key)
        hf.create_dataset(key,  data= meta_data[key][:])

  0%|          | 0/11 [00:00<?, ?it/s]

ecog_alldataset
label_alldataset
formant_re_alldataset
pitch_re_alldataset
intensity_re_alldataset
start_ind_re_valid_alldataset
end_ind_re_valid_alldataset
wave_re_spec_alldataset
wave_re_alldataset
wave_re_spec_amp_alldataset
noisesample_re_alldataset
